In [1]:
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline
import random

In [2]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, RNN, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import pandas as pd
np.random.seed(1)

Using TensorFlow backend.


### Helper functions for Data Loading, normalization, etc

In [3]:
def Reshape_to_input(X, T, Mat_dim):
    """
    reshapes data
    """
   
    [Num_examples, dim] = X.shape
    input_X = X[:,2:].reshape(Num_examples, Mat_dim, T)
    
    return input_X
    

In [4]:
def NormalizeTowardsMean(X):
   # print(X.shape)
    "# Preprocessing: Subtract the mean feature - a question if it is feasible - all features are already in the same scale...\n",
    # TODO: try -1 1 normalization
    mean_feat = np.mean(X, axis=0, keepdims=True)
   # print(mean_feat.shape)
    X -= mean_feat
    # Preprocessing: Divide by standard deviation. This ensures that each feature\n",
    # has roughly the same scale.\n",
    std_feat = np.std(X, axis=0, keepdims=True)
    X /= std_feat
    
    return X



In [5]:
def Random_Selection_train_test (X_all, N):
#    import random
    " This fucntion takes return train and test samples for the LSTM: X dataset, N - number of training samples (i.e persons)"
    Persons = X_all[:,1]
    uniquePersons = np.unique(Persons)
    random.shuffle(uniquePersons)
    num_dim = np.shape(X_all)[1]
    X_train = np.array([], dtype=np.int64).reshape(0,num_dim)
    Y_train = np.array([])
    X_test = np.array([])
    Y_test = np.array([])
    Persons_train = np.array([])
    Persons_test = np.array([])
    #print(X_train.shape)
    for Person in range(0,N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_train= np.append(X_train,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_train = np.append(Y_train, X_all[[indexes], 0])
        Persons_train = np.append(Persons_train, label, axis = 0)
    X_train=X_train.reshape(Y_train.size, num_dim)    
    for Person in range(N,uniquePersons.size):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        X_test = np.append(X_test, X_all[[indexes], :][0,0,:,:])
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_test =  np.append(Y_test, X_all[[indexes], 0])
        Persons_test = np.append(Persons_test, label, axis = 0)
    X_test=X_test.reshape(Y_test.size, num_dim) 
    print('Persons for train: ')
    print(np.unique(Persons_train))
    print('Persons for test: ')
    print(np.unique(Persons_test))
    return X_train, Y_train, X_test, Y_test, Persons_train
            
            


In [6]:
def Random_Selection_train_val (X, Y, N, Person_labels):
    uniquePersons = np.unique(Person_labels)
    random.shuffle(uniquePersons)
    num_dim = np.shape(X)[1]
    X_train = np.array([])
    Y_train = np.array([])
    X_val = np.array([])
    Y_val = np.array([])
    Persons_train = np.array([])
    Persons_val = np.array([])
    for Person in range(0,N):
        indexes = np.where(Person_labels==uniquePersons[Person])
        Person_X_val = X[[indexes], :]
        X_train= np.append(X_train,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_train = np.append(Y_train, Y[indexes])
        Persons_train = np.append(Persons_train, label, axis = 0)
    X_train=X_train.reshape(Y_train.size, num_dim) 
    for Person in range(N,uniquePersons.size):
        indexes = np.where(Person_labels==uniquePersons[Person])
        Person_X_val = X[[indexes], :]
        X_val= np.append(X_val,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_val = np.append(Y_val, Y[indexes])
        Persons_val = np.append(Persons_val, label, axis = 0)
    X_val=X_val.reshape(Y_val.size, num_dim) 
    print('Persons for train: ')
    print(np.unique(Persons_train))
    print(Y_train)
    print('Persons for validation: ')
    print(np.unique(Persons_val))
    print(Y_val)
    print(X_train.shape)
    print(X_val.shape)
    return X_train, Y_train, X_val, Y_val

In [7]:
def Random_Selection_train_val_test (X_all, N):
#    import random
    " This fucntion takes return train and test samples for the LSTM: X dataset, N - number of training samples (i.e persons)"
    Persons = X_all[:,1]
    uniquePersons = np.unique(Persons)
    random.shuffle(uniquePersons)
    num_dim = np.shape(X_all)[1]
    X_train = np.array([], dtype=np.int64).reshape(0,num_dim)
    Y_train = np.array([])
    X_test = np.array([])
    Y_test = np.array([])
    X_val = np.array([])
    Y_val = np.array([])
    Persons_train = np.array([])
    Persons_val = np.array([])
    Persons_test = np.array([])
    
    train_N =int(np.ceil(N*2/3))
    
    for Person in range(0,train_N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_train= np.append(X_train,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_train = np.append(Y_train, X_all[[indexes], 0])
        Persons_train = np.append(Persons_train, label, axis = 0)
    X_train=X_train.reshape(Y_train.size, num_dim)    
    
     
    for Person in range(train_N,N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_val= np.append(X_val,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_val = np.append(Y_val, X_all[[indexes], 0])
        Persons_val = np.append(Persons_val, label, axis = 0)
    X_val=X_val.reshape(Y_val.size, num_dim)  
    
    for Person in range(N,uniquePersons.size):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        X_test = np.append(X_test, X_all[[indexes], :][0,0,:,:])
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_test =  np.append(Y_test, X_all[[indexes], 0])
        Persons_test = np.append(Persons_test, label, axis = 0)
    X_test=X_test.reshape(Y_test.size, num_dim) 
    print('Persons for train: ')
    print(np.unique(Persons_train))
    returned_train = np.unique(Persons_train)
    print('Persons for val: ')
    print(np.unique(Persons_val))
    returned_val = np.unique(Persons_val)
    print('Persons for test: ')
    print(np.unique(Persons_test))
    return X_train, Y_train, X_test, Y_test, X_val, Y_val, returned_train, returned_val
            
            


In [8]:
def convert_to_one_hot(inputY, C):
    N= inputY.size
    Y=np.zeros((N,C))
    for i in range (0, inputY.size):
        Y[i, int(inputY[i]-1)] = 1
        
    
    return Y


In [9]:
def Gait_model(data):
    """
    Function creating the Gait model's graph.
    
    Arguments:
    input - data

    Returns:
    model -- a model instance in Keras
    """
   
    ### START CODE HERE ###
    gait_data = Input(data.shape, dtype='float32')
    units = 128 
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.

    X = Bidirectional(LSTM(units, return_sequences=True))( gait_data)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
   
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    X = Bidirectional(LSTM(units, return_sequences=False))(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(3)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=gait_data, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [10]:
# load my dataset and parameters from a file
from data_utils import loadfromfolder

### Model Training

In [11]:
myoptim=Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

def get_callbacks(filepath, patience=2):
    es = EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='min')
    mcp_save = ModelCheckpoint(filepath, save_best_only =True, monitor = 'val_loss', mode ='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, verbose=1, epsilon=1e-4, mode='min') #epsilon
    return [es, mcp_save, reduce_lr_loss]

In [12]:
# parameters needed
trials = 1
epochs = 25
batch_size = 25
units = 192 # actually defind inside the funciton of the model - TODO - make it here
path ='data/best'
Files, T, max_dim = loadfromfolder()
#Where Files are names, T - my division into frames and max_dim - feature dimensionality
#fRes = open("Results.txt","a+") 
#fRes.write("Number of units in LSTM = %d,  Epochs =  %d and batch size =  %d \r\n" % (units, epochs,batch_size))
for file in range(0, len(Files)):
    name = Files[file]
    t = T[file]
    dim = max_dim[file]
    name_w_path = path +"/"+name
    #fRes.write("Experiment %d from file %s \r\n" % (file,name))
    # here all the routine in One cell 
    X_all = pd.read_csv(name_w_path)
    # the routine to run the same test N times, randomly shuffling the data
    Accuracy  = np.zeros(shape=(trials))
    X_train_all, Y_train_all, X_test, Y_test, returned_train = Random_Selection_train_test (X_all.values, 16)
    Y_test_oh = convert_to_one_hot(Y_test, C = 3)
    X_test = Reshape_to_input(X_test, t, dim)
    for i in range(0,trials):
        modelname = str(file)+str(i)
        
        X_train, Y_train, X_val, Y_val, returned_train = Random_Selection_train_test (X_train_all, 11)

        Y_train_oh = convert_to_one_hot(Y_train, C = 3)
        Y_val_oh = convert_to_one_hot(Y_val, C = 3)
        
        X_train = Reshape_to_input(X_train, t, dim)
        X_val = Reshape_to_input(X_val, t, dim)
        
        
        model = Gait_model((X_train[0,:,:]))
        
        file_path = 'models/file_{}_model_wts.hdf5'.format(modelname)
        callbacks = get_callbacks(filepath=file_path, patience = 35)
        model.compile(loss='categorical_crossentropy', optimizer=myoptim, metrics=['accuracy'])
              
              
        
        model.fit(X_train, Y_train_oh, epochs=epochs,\
                  batch_size=batch_size, shuffle=True, validation_data=(X_val, Y_val_oh), callbacks=callbacks)
        
        loss, acc = model.evaluate(X_test, Y_test_oh)
        
        probability_assigned = model.predict(X_test) #verbose=1
        print(probability_assigned)
        Accuracy[i]=acc
        #fRes.write("Final accuracy per trial %f \r\n" % (acc))
        
    print(Accuracy)
    #fRes.write("Final mean accuracy is: %f \r\n" % (np.mean(Accuracy)))
#fRes.close() 


angles_cycles_frames_15t_08f_angles_22persons.csv
Persons for train: 
[ 1.  2.  4.  5.  6.  7.  9. 10. 12. 13. 14. 15. 16. 18. 19. 21.]
Persons for test: 
[ 3.  8. 11. 17. 20. 22.]
Persons for train: 
[ 4.  5.  7.  9. 10. 12. 13. 14. 16. 18. 21.]
Persons for test: 
[ 1.  2.  6. 15. 19.]


/home/khokhlov/virtenv/ml36/lib/python3.5/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 283 samples, validate on 115 samples
Epoch 1/25
283/283 [==============================] - 7s 25ms/step - loss: 1.0285 - acc: 0.4488 - val_loss: 0.9573 - val_acc: 0.5913
Epoch 2/25
283/283 [==============================] - 1s 3ms/step - loss: 0.8814 - acc: 0.6572 - val_loss: 0.8396 - val_acc: 0.6435
Epoch 3/25
283/283 [==============================] - 1s 3ms/step - loss: 0.7426 - acc: 0.6996 - val_loss: 0.7399 - val_acc: 0.6870
Epoch 4/25
283/283 [==============================] - 1s 3ms/step - loss: 0.6439 - acc: 0.7703 - val_loss: 0.6569 - val_acc: 0.7652
Epoch 5/25
283/283 [==============================] - 1s 3ms/step - loss: 0.5422 - acc: 0.8057 - val_loss: 0.5944 - val_acc: 0.7826
Epoch 6/25
283/283 [==============================] - 1s 3ms/step - loss: 0.5109 - acc: 0.7986 - val_loss: 0.5507 - val_acc: 0.7478
Epoch 7/25
283/283 [==============================] - 1s 3ms/step - loss: 0.4450 - acc: 0.8516 - val_loss: 0.5254 - val_acc: 0.7652
Epoch 8/25
283/283 [=========

AttributeError: 'Model' object has no attribute 'predict_proba'

In [17]:
print(model.predict(X_test))

[[9.98949707e-01 8.70735676e-04 1.79563649e-04]
 [9.98317719e-01 1.54931098e-03 1.33060967e-04]
 [9.98885572e-01 8.92005977e-04 2.22434799e-04]
 [9.98817742e-01 9.85471415e-04 1.96671070e-04]
 [9.99099016e-01 7.25850288e-04 1.75211579e-04]
 [9.95639563e-01 3.58978566e-03 7.70696439e-04]
 [2.67765630e-04 9.93724167e-01 6.00807322e-03]
 [3.91293142e-04 9.95800793e-01 3.80787882e-03]
 [5.32922801e-04 9.98729765e-01 7.37328839e-04]
 [6.88194923e-05 9.97945964e-01 1.98531384e-03]
 [1.93786123e-04 9.97772992e-01 2.03323714e-03]
 [1.29415275e-04 9.99122918e-01 7.47596147e-04]
 [1.14455152e-05 3.24362703e-02 9.67552304e-01]
 [4.77309504e-06 6.94034342e-03 9.93054926e-01]
 [2.81091707e-05 1.77904710e-01 8.22067201e-01]
 [4.12577583e-06 1.01512773e-02 9.89844561e-01]
 [1.90592004e-06 7.24747404e-03 9.92750645e-01]
 [3.62772789e-06 8.73268023e-03 9.91263747e-01]
 [5.44257944e-07 6.81274279e-04 9.99318242e-01]
 [9.92303908e-01 5.87433856e-03 1.82169257e-03]
 [9.98090684e-01 1.18078059e-03 7.284752

In [ ]:
fRes.close() 